# Evaluation of Different Models

In [1]:
import json
import scipy
import torch

import numpy as np
import pandas as pd

from src.models import classification_transformer

In [2]:
embedding_transformer = classification_transformer.Main("large")

In [3]:
data = pd.read_csv('../Data/data_tokenized_pitch_class.csv')
data["chords"] = data["chords"].apply(lambda x: torch.tensor(json.loads(x)))
data.head()

,url,title,artist,decade,genre,ratings,stars,chords
0,https://tabs.ultimate-guitar.com/tab/jeff-buck...,Hallelujah,Jeff Buckley,1990,Rock|Folk,51639.0,5.0,"[tensor(648), tensor(640), tensor(648), tensor..."
1,https://tabs.ultimate-guitar.com/tab/ed-sheera...,Perfect,Ed Sheeran,2010,Pop,44194.0,5.0,"[tensor(155), tensor(31), tensor(648), tensor(..."
2,https://tabs.ultimate-guitar.com/tab/elvis-pre...,Cant Help Falling In Love,Elvis Presley,1960,"Soundtrack|R&B, Funk & Soul",30059.0,5.0,"[tensor(648), tensor(155), tensor(640), tensor..."
3,https://tabs.ultimate-guitar.com/tab/eagles/ho...,Hotel California,Eagles,1970,Rock,28670.0,5.0,"[tensor(166), tensor(403), tensor(380), tensor..."
4,https://tabs.ultimate-guitar.com/tab/radiohead...,Creep,Radiohead,1990,Rock,28606.0,5.0,"[tensor(155), tensor(78), tensor(648), tensor(..."


In [4]:
# Split the data into train and test sets as in the previous notebooks
import torch
from torch.utils.data import random_split

torch.manual_seed(42)

class ChordDataset: # A dummy dataset class
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

dataset = ChordDataset(data)
train_size = int(np.rint(len(dataset) * 0.8))
train_indices, test_indices = random_split(range(len(dataset)), [train_size, len(dataset) - train_size])

# Convert the indices to lists
train_indices = [idx for idx in train_indices.indices]
test_indices = [idx for idx in test_indices.indices]

# Split the dataframe using the indices
train_data = data.iloc[train_indices]
test_data = data.iloc[test_indices]

# Reindex the dataframes
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [5]:
print(f"Train size: {len(train_data)}, Test size: {len(test_data)}")

Train size: 17792, Test size: 4448


## Tester

In [6]:
class Evaluation:
    def __init__(self, ref_column):
        self.ref_column = ref_column        
        self.augmentation_map = torch.tensor(np.load('../Data/augmentation_map.npy', allow_pickle=True))
        self.augmentation_map = self.augmentation_map.to(embedding_transformer.device)

    
    def augment(self, chords):
        """Change the root note of the chords by a random amount"""
        move_by = torch.randint(0, 12, [1]).item()
        return self.augmentation_map[chords, move_by]
        
    def pad(self, chords):
        """Pad the input 2D tensor [n] into shape [256] with zeros"""
        out = torch.zeros((256), dtype=torch.long, device=embedding_transformer.device)
        out[:len(chords)] = chords
        return out
    
    def reduce_dimensionality(self, column, batch_size=64, augment=False):
        """Use a pretrained classifier to reduce the dimensionality of the dataframes."""
        
        reduced_column = []
        n = len(column)
        
        for i in range(0, n, batch_size):
            # Extract the current batch of data
            batch = column[i:i+batch_size]
            
            # Convert batch to torch tensor and process it
            batch_tensor = [torch.tensor(item.tolist(), dtype=torch.long, device=embedding_transformer.device) for item in batch]
            if augment:
                batch_tensor = torch.stack([self.pad(self.augment(item)) for item in batch_tensor])
            else:
                batch_tensor = torch.stack([self.pad(item) for item in batch_tensor])
            
            # Get embeddings for the entire batch and append to reduced_column
            batch_embeddings = embedding_transformer.batch_extract_features(batch_tensor)
            reduced_column.extend(batch_embeddings)
            
        return reduced_column
    
    def frechet_distance(self, mu1, mu2, sigma1, sigma2):
        """
        Compute the Frechet distance between two multivariate Gaussians.
        
        Args:
            mu1, mu2: mean vectors (1D numpy arrays)
            sigma1, sigma2: covariance matrices (2D numpy arrays)
            
        Returns:
            The Frechet distance between the two distributions.
        """
        mu_diff = mu1 - mu2
        # The following line computes (sigma1 * sigma2)^(1/2) using the matrix square root
        sqrt_sigma = scipy.linalg.sqrtm(np.dot(sigma1, sigma2))
        
        # Handling numerical instability (may occur if matrices are nearly singular)
        if not np.isfinite(sqrt_sigma).all():
            offset = np.eye(sigma1.shape[0]) * 1e-10
            sqrt_sigma = scipy.linalg.sqrtm(np.dot(sigma1 + offset, sigma2 + offset))
        
        # Compute the trace term
        tr_term = np.trace(sigma1 + sigma2 - 2 * sqrt_sigma)
        
        # Compute the difference term
        diff_term = np.dot(mu_diff, mu_diff)
        
        return diff_term + tr_term
    
    def calculate_frechet_distance(self, ref_column, gen_column):
        """Calculate the Frechet distance between the reference and generated samples for the reduced columns."""
        ref_column, gen_column = np.array(ref_column), np.array(gen_column)
        mu1 = np.mean(ref_column, axis=0)
        mu2 = np.mean(gen_column, axis=0)

        sigma1 = np.cov(ref_column, rowvar=False)
        sigma2 = np.cov(gen_column, rowvar=False)

        # Compute the Frechet distance
        return self.frechet_distance(mu1, mu2, sigma1, sigma2)
    
    def preprocess_ref_col(self):
        """Reduce the dimensionality of the reference column."""
        self.ref_reduced_column = self.reduce_dimensionality(self.ref_column, augment=True)
    
    def get_column_score(self, gen_column):
        """Get the score for a generated column."""
        gen_reduced_column = self.reduce_dimensionality(gen_column)
        distances = self.calculate_frechet_distance(self.ref_reduced_column, gen_reduced_column)
        return distances

In [7]:
torch.manual_seed(42)
eval = Evaluation(test_data["chords"])
eval.preprocess_ref_col()

d:\Programs\anaconda3\lib\site-packages\torch\nn\modules\activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\attention.cpp:152.)
  return torch._native_multi_head_attention(


In [8]:
def evaluate(file_name):
    gen_df = pd.read_csv(f"../Data/Generated/{file_name}.csv")
    gen_df["chords"] = gen_df["chords"].apply(lambda x: torch.tensor(json.loads(x), dtype=torch.long))
    score = eval.get_column_score(gen_df["chords"])
    return score

In [9]:
scores = []
sizes = ["small", "medium", "large"]
files = ["recurrent_net"] + [f"transformer_{s}" for s in sizes] + [f"conditional_{s}" for s in sizes] + [f"style_{s}" for s in sizes]
for file_name in files:
    scores.append({"model": file_name, "score": evaluate(file_name)})
scores = pd.DataFrame(scores)
scores

d:\Programs\anaconda3\lib\site-packages\torch\nn\modules\activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\attention.cpp:152.)
  return torch._native_multi_head_attention(
d:\Programs\anaconda3\lib\site-packages\torch\nn\modules\activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\attention.cpp:152.)
  return torch._native_multi_head_attention(
d:\Programs\anaconda3\lib\site-packages\torch\nn\modules\activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at C

,model,score
0,recurrent_net,9.474979
1,transformer_small,3.882284
2,transformer_medium,2.549732
3,transformer_large,2.021196
4,conditional_small,3.934798
5,conditional_medium,1.935170
6,conditional_large,1.424958
7,style_small,2.189804
8,style_medium,0.970757
9,style_large,0.705609
